In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy.optimize as sci_opt
import yfinance as yf

import sys
import os
print(os.chdir("/home/am_yss/python/portfolio_optimizer_project/"))

print(os.getcwd())


None
/home/am_yss/python/portfolio_optimizer_project


In [21]:
from backend.data.fetch_data import DataFetcher

tickers = ["2222.SR", "1120.SR",
              "1180.SR", "2010.SR",
              "7010.SR", "1211.SR",
              "1010.SR", "1140.SR",
              "1150.SR", "5110.SR",
              "TSLA", "URTH"]



fetcher = DataFetcher(tickers = tickers, years = 15, endDate= dt.datetime.now())

df= fetcher.fetch_data()

market_data = df['URTH']

df = df.drop(columns = ['URTH'])

display(df.head())

display(market_data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/home/am_yss/python/portfolio_optimizer_project/backend/data/fetch_data.py:50: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future versio

,2222.SR,1120.SR,1180.SR,2010.SR,7010.SR,1211.SR,1010.SR,1140.SR,1150.SR,5110.SR,TSLA
Date,,,,,,,,,,,
2019-12-11,24.697298,35.565033,29.023945,72.775513,31.580549,13.750000,19.852274,15.258702,12.953329,17.063610,23.513332
2019-12-12,25.819906,34.949909,28.322741,70.344437,31.104456,13.533333,19.428152,15.117681,12.783170,16.995626,23.978666
2019-12-15,26.240877,34.949909,28.200787,70.658119,31.421843,13.550000,19.509710,15.315111,13.612694,17.029621,23.978666
2019-12-16,26.661854,35.620949,28.475178,71.285492,31.453587,13.466666,19.803337,15.286908,13.825392,16.944641,25.433332
2019-12-17,26.486452,35.956474,28.871508,70.814964,32.120113,13.666666,19.167150,15.315111,13.825392,17.352535,25.266001


Date
2019-12-11     88.701630
2019-12-12     89.249466
2019-12-15     89.249466
2019-12-16     90.374657
2019-12-17     90.107391
                 ...    
2025-03-04    156.960007
2025-03-05    159.240005
2025-03-06    156.490005
2025-03-09    156.490005
2025-03-10    153.339996
Name: URTH, Length: 1311, dtype: float64

In [27]:
#Calculate the log returns

log_returns = np.log1p(df.pct_change()).iloc[1:]
market_returns = np.log1p(market_data.pct_change().iloc[1:])

#setup the random weights and rebalance them

random_weights = np.array(np.random.random(df.columns.nunique()))




In [120]:
#risk free rate
import requests as r
API_KEY = "802b7806b74b56166b494d10a42a01c5"
FRED_ENDPOINT = "https://api.stlouisfed.org/fred/series/observations"
FILE_TYPE = "json"
SERIES_ID = "DGS10"

fred_info = {"api_key": API_KEY,
             "file_type": FILE_TYPE,
             "series_id": SERIES_ID,
             "observation_start": (dt.datetime.now() - dt.timedelta(days = 365*6)).strftime(f"%Y-%m-%d"),
             "observation_end": dt.datetime.now().strftime(f"%Y-%m-%d"),
             }

response = r.get(FRED_ENDPOINT, params = fred_info)

data = response.json()

risk_df = pd.DataFrame(data["observations"])
risk_df["date"] = pd.to_datetime(risk_df["date"])  # Convert date column to datetime format
risk_df["value"] = risk_df["value"]
risk_df = risk_df.iloc[1:]

display(risk_df.head())

display(risk_df.dtypes)

,realtime_start,realtime_end,date,value
1,2025-03-11,2025-03-11,2019-03-15,2.59
2,2025-03-11,2025-03-11,2019-03-18,2.60
3,2025-03-11,2025-03-11,2019-03-19,2.61
4,2025-03-11,2025-03-11,2019-03-20,2.54
5,2025-03-11,2025-03-11,2019-03-21,2.54


realtime_start            object
realtime_end              object
date              datetime64[ns]
value                     object
dtype: object

In [121]:
try:
    risk_df = risk_df.drop(["realtime_start", "realtime_end"], axis = 1)
except KeyError:
    print(f"Series not found in axis = 1")

risk_df["value"] = pd.to_numeric(risk_df["value"], errors = "coerce")

print(risk_df.dtypes)

risk_df.head()


date     datetime64[ns]
value           float64
dtype: object


,date,value
1,2019-03-15,2.59
2,2019-03-18,2.60
3,2019-03-19,2.61
4,2019-03-20,2.54
5,2019-03-21,2.54


In [122]:
risk_df[risk_df['date'] == "2019-12-12"].index

risk_df = risk_df.iloc[193:]

risk_df.head()

,date,value
194,2019-12-11,1.79
195,2019-12-12,1.90
196,2019-12-13,1.82
197,2019-12-16,1.89
198,2019-12-17,1.89


In [123]:
weights = random_weights / sum(random_weights)

sum(weights)

np.float64(0.9999999999999999)

### **Historical Mean Returns Method**


## Definition
The **Historical Mean Returns Method** calculates the average past returns of an asset or portfolio to estimate its expected return. This approach assumes that historical performance is indicative of future returns.

## Formula
The historical mean return is calculated as:

$$ R_{mean} = \frac{1}{N} \sum_{i=1}^{N} R_i $$

Where:
- \( R_{mean} \) = Historical mean return
- \( R_i \) = Return in period \( i \)
- \( N \) = Total number of periods

## Volatility (Standard Deviation of Portfolio Returns)
The volatility of a portfolio is given by:

$$ \sigma_p = \sqrt{ w^T \cdot (\Sigma \cdot n) \cdot w } $$

Where:
Where:
- \( \sigma_p \) = Portfolio volatility (annualized)
- \( w \) = Vector of asset weights in the portfolio
- \( \Sigma \) = Covariance matrix of log returns
- \(  n \) = Number of trading days in a year

## Sharpe Ratio
The **Sharpe Ratio** measures risk-adjusted return and is defined as:

$$ S = \frac{R_p - R_f}{\sigma_p} $$

Where:
- \( S \) = Sharpe ratio
- \( R_p \) = Portfolio return
- \( R_f \) = Risk-free rate
- \( \sigma_p \) = Portfolio standard deviation (volatility)


In [135]:
historical_mean_return = np.dot(log_returns.mean(),  weights) * 252

historical_returns = np.dot(log_returns,  weights) * 252


volatility = np.sqrt(
    np.dot(
        weights.T,
        np.dot(
            log_returns.cov() *252,
            weights
        )
    )
)

risk_free_rate = (1 + risk_df['value'].mean()) ** (1/252) - 1 #daily

sharpe_ratio = (historical_mean_return - risk_free_rate)/volatility

display(risk_free_rate)
display(sharpe_ratio)

display(historical_mean_return)



np.float64(0.005251760477749556)

np.float64(0.5664353939362994)

np.float64(0.11280355201186335)

# Capital Asset Pricing Model (CAPM)

## Definition
The **Capital Asset Pricing Model (CAPM)** is used to determine the expected return of an asset given its systematic risk (beta) relative to the market.

## Formula

$$ R_i = R_f + \beta_i (R_m - R_f) $$

Where:
- \( R_i \) = Expected return of asset \( i \)
- \( R_f \) = Risk-free rate
- \( \beta_i \) = Beta of asset \( i \) (sensitivity to market movements)
- \( R_m \) = Expected return of the market
- \( R_m - R_f \) = Market risk premium

## Beta Calculation
Beta (\( \beta \)) is calculated as:

$$ \beta_i = \frac{Cov(R_i, R_m)}{Var(R_m)} $$

Where:
- \( Cov(R_i, R_m) \) = Covariance of asset return with market return
- \( Var(R_m) \) = Variance of market return



We will use the CAPM to compare with the historical mean returns and determine if the portfolio is earning more than the market adjusted expectation.

### Why is the CAPM useful?

* The CAPM is useful because it allows the user to check if their portfolio is underperforming or overperforming relative to the market.
* It allows Jensen's alpha computation to test whether the portfolio is generating statstically significant excess returns
    -  if **alpha > 0**, the portfolio is generating returns above market expectations.
    - if **alpha < 0**, the portfolio is underperforming relative to risk-adjusted expectations.

In [137]:
capm_list = []
for ticker in log_returns.columns:
    #covariance matrix
    cm = np.cov(log_returns[ticker], market_returns)
    beta = cm[0, 1]/cm[1,1]
    #expected market returns
    e_mr = market_returns.mean() * 252
    #expected returns
    er = risk_free_rate + beta * (e_mr - risk_free_rate)
    alpha_test =  historical_mean_return - (risk_free_rate + beta * (e_mr - risk_free_rate))
    capm_list.append({'Asset': ticker,
                      "CAPM_returns": er,
                      "Beta": beta,
                      "Jensens-test": alpha_test,
                      })



capm_df = pd.DataFrame(capm_list).set_index("Asset")

capm_df['weight'] = weights

capm_df['weighted_alphas'] = capm_df['Jensens-test'] * capm_df['weight']

display(capm_df)
portfolio_capm_returns = np.dot(capm_df["CAPM_returns"], weights)

print(portfolio_capm_returns)

portfolio_alpha = capm_df['weighted_alphas'].sum()

print(portfolio_alpha)

,CAPM_returns,Beta,Jensens-test,weight,weighted_alphas
Asset,,,,,
2222.SR,0.021407,0.161477,0.091397,0.039135,0.003577
1120.SR,0.027923,0.226604,0.084881,0.105669,0.008969
1180.SR,0.031821,0.265575,0.080982,0.076897,0.006227
2010.SR,0.027375,0.221136,0.085428,0.154420,0.013192
7010.SR,0.015973,0.107168,0.096830,0.052584,0.005092
1211.SR,0.039485,0.342178,0.073318,0.182229,0.013361
1010.SR,0.028985,0.237219,0.083819,0.175603,0.014719
1140.SR,0.028383,0.231211,0.084420,0.031797,0.002684
1150.SR,0.031009,0.257458,0.081794,0.064083,0.005242


0.036071669428164296
0.07673188258369902


### Value at Risk

In [72]:
confidence_level = 0.95

VaR_historical = -np.percentile(log_returns, (1 - confidence_level)* 100)

print(f"Historical VaR (95% confidence level): {VaR_historical:.2%}")



Historical VaR (95% confidence level): 2.66%
